# Figure 1: Example Radiology Report

In [1]:
from pathlib import Path

import pandas as pd

from guidedsum.evaluation import load_summaries

In [2]:
def best_step(run_path):
    run_path = Path(run_path)
    try:
        with open(run_path / 'model_step_best.txt') as fin:
            return int(fin.read().strip())
    except FileNotFoundError:
        return -1

In [3]:
STEP_BERTABS = best_step("../output/mimic-official-unguided/bertabs-default/")
STEP_GSUM = best_step('../output/mimic-official-oracle/gsum-default/')

runs = [
    ("../output/mimic-official-unguided/bertabs-default/", STEP_BERTABS, "bertabs"),
    ("../output/mimic-official-bertext-default-clip-k1/gsum-default/", STEP_GSUM, "gsum_fixed"),
    ("../output/mimic-official-bertext-default-clip-threshold/gsum-default/", STEP_GSUM, "gsum_thresholding"),
]


candidates = []
for run_path, step, name in runs:
    df_summaries = load_summaries(run_path, step)
    c = df_summaries['candidate'].rename(name)
    candidates.append(c)
       
df_reports = pd.read_json("../data/processed/mimic-official/reports.test.json")
df_candidates = pd.concat(candidates, axis=1)

df = pd.concat([df_reports, df_candidates], axis=1)
df.head(2)

,impression,findings,comparison,examination,history,technique,indication,id,findings+bg,chexpert_labels_findings,chexpert_labels_impression,bertabs,gsum_fixed,gsum_thresholding
0,No evidence of acute cardiopulmonary process.,PA and lateral views of the chest are compared...,,,_-year-old male with cough and history of HIV....,,,50010466,History:\n_-year-old male with cough and histo...,"{'No Finding': None, 'Enlarged Cardiomediastin...","{'No Finding': 1.0, 'Enlarged Cardiomediastinu...",no acute cardiopulmonary process .,no acute cardiopulmonary process .,no acute cardiopulmonary process .
1,"Vague bibasilar opacities, which may represent...",Frontal view of the chest was obtained. A righ...,"Multiple prior chest radiographs, most recentl...",,,,_-year-old male with HIV with shaking chills a...,50014127,Indication:\n_-year-old male with HIV with sha...,"{'No Finding': None, 'Enlarged Cardiomediastin...","{'No Finding': None, 'Enlarged Cardiomediastin...",vague bibasilar opacities are nonspecific but ...,vague bibasilar opacities are nonspecific but ...,vague bibasilar opacities are nonspecific but ...


In [4]:
sample = df[df['id'] == 53203970].iloc[0]
print(sample['findings+bg'])

print('\nBertAbs (unguided)')
print(sample['bertabs'])

print('\nGSum Fixed')
print(sample['gsum_fixed'])

print('\nGSum Variabe')
print(sample['gsum_thresholding'])

Technique:
Chest, AP and lateral.

Comparison:
_ and _.

History:
Weakness and decreased blood sugar with leg swelling and tenderness.

Findings:
The patient is status post coronary artery bypass graft surgery and apparently mitral valve replacement. The heart is mildly enlarged. The mediastinal and hilar contours appear unchanged. There is a slight interstitial abnormality, suggestive of a state of very mild congestion, but no new focal opacity. A left-sided pleural effusion has resolved although mild scarring or atelectasis persists. Bones are probably demineralized.

BertAbs (unguided)
findings suggesting mild vascular congestion .

GSum Fixed
findings suggest mild vascular congestion .

GSum Variabe
findings suggest mild vascular congestion .<q>resolution of left - sided pleural effusion .


In [5]:
z_fixed = pd.read_json('../data/processed/mimic-official-bertext-default-clip-k1/reports.test.json')
print(z_fixed[z_fixed['id'] == 53203970].iloc[0]['z'])

[['There', 'is', 'a', 'slight', 'interstitial', 'abnormality', ',', 'suggestive', 'of', 'a', 'state', 'of', 'very', 'mild', 'congestion', ',', 'but', 'no', 'new', 'focal', 'opacity', '.']]


In [6]:
z_variable = pd.read_json('../data/processed/mimic-official-bertext-default-clip-threshold/reports.test.json')
print(z_variable[z_variable['id'] == 53203970].iloc[0]['z'])

[['There', 'is', 'a', 'slight', 'interstitial', 'abnormality', ',', 'suggestive', 'of', 'a', 'state', 'of', 'very', 'mild', 'congestion', ',', 'but', 'no', 'new', 'focal', 'opacity', '.'], ['A', 'left', '-', 'sided', 'pleural', 'effusion', 'has', 'resolved', 'although', 'mild', 'scarring', 'or', 'atelectasis', 'persists', '.']]
